In [61]:
import pandas as pd
import lifelines
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter
from sklearn.preprocessing import OneHotEncoder


In [62]:
df = pd.read_csv('FHS_Network_features_dementia_risk_dataforsurvivalanalysis_wave7.csv',index_col=0).reset_index(drop=True)




In [63]:
network_features = ['DEGREE',
                    'DEGREE_CENTRALITY', 
                    'CONSTRAINT',
                    'EFFECTIVE_SIZE',
                    'EIGENVECTOR',
                    'KATZ',
                    'CLOSENESS_CENTRALITY',
                    'BETWEENNESS_CENTRALITY',
                    'CLUSTCOEF',
                    'AGE7',
                    'SEX',
                    'somecoll_education',
                    'HS_education',
                    'APOE_risk'
                   ]

modifiers = [c for c in df.columns if 'SHANNON' in c or 'SIMPSON' in c]



In [64]:
df.columns

Index(['idr', 'SEX', 'AGE7', 'BMI7', 'CURRSMK7', 'SBP7', 'DBP7', 'CALC_LDL7',
       'HDL7', 'TC7', 'TRIG7', 'DATE7', 'idtype', 'DEM_STATUS', 'DEM_SURVDATE',
       'AD_STATUS', 'VAD_STATUS', 'DENSITY', 'CONSTRAINT', 'EFFECTIVE_SIZE',
       'DEGREE', 'DEGREE_CENTRALITY', 'EIGENVECTOR', 'KATZ',
       'CLOSENESS_CENTRALITY', 'BETWEENNESS_CENTRALITY', 'CLUSTCOEF', 'APOE',
       'APOE_risk', 'CURR_DIAB7', 'Death_status_at_followup',
       'somecoll_education', 'HS_education', 'education_4level', 'CES-D_score',
       'CSD-D_loneliness_item', 'SEX_SHANNON', 'SEX_SIMPSON',
       'RELTYPE_SHANNON', 'RELTYPE_SIMPSON', 'ALTERTYPE_SHANNON',
       'ALTERTYPE_SIMPSON', 'AGE35_SHANNON', 'AGE35_SIMPSON', 'AGE65_SHANNON',
       'AGE65_SIMPSON', 'AGE_Q_SHANNON', 'AGE_Q_SIMPSON', 'AGE_10_SHANNON',
       'AGE_10_SIMPSON', 'CESD_16_SHANNON', 'CESD_16_SIMPSON',
       'CESD_20_SHANNON', 'CESD_20_SIMPSON', 'LONELY_3_SHANNON',
       'LONELY_3_SIMPSON', 'SNI_2_SHANNON', 'SNI_2_SIMPSON',
       'EDU_

In [65]:
##for diversity indices, change anything >0 as 1
for c in modifiers:
    df[c] = df[c].apply(lambda x: 1 if x>0 else 0)
    
    

In [66]:
[modifiers.append(x) for x in ['SEX','AGE7','APOE_risk','somecoll_education','HS_education','CVD_HISTORY','CES-D_score','CSD-D_loneliness_item']]


[None, None, None, None, None, None, None, None]

In [67]:
# Define Event and time.to.event variables
df['event'] = df['DEM_STATUS']
df['time_to_event'] = df['DEM_SURVDATE']

df_cox = df[['event','time_to_event']]


In [68]:
#keep event as dem status
# df_COX <- within(df_COX, Event[Event == 0 & Death_status_at_followup == 1] <- 0)
# # Same for anyone who did  NOT die or did NOT have dementia.  
# df_COX <- within(df_COX, Event[Event == 0 & Death_status_at_followup == 0] <- 0)
# # Anyone with dementia (regardless of death status) is coded as an event (1).
# df_COX <- within(df_COX, Event[Event == 1 & Death_status_at_followup == 1] <- 1)
# df_COX <- within(df_COX, Event[Event == 1 & Death_status_at_followup == 0] <- 1)

In [69]:

cph = CoxPHFitter()

results = []
for f in network_features:
    for m in modifiers:
        df_cox['interaction_term'] = df[f]*df[m]
        try:
            cph.fit(df_cox, duration_col='time_to_event', event_col='event')
            res= cph.summary.reset_index().replace({'interaction_term':f+'*'+m})
            res['term 1'] = f
            res['term 2'] = m
            results.append(res)
            if  cph.summary['p'].values[0]<.05:
                print('-'*50)
                print(f+' * '+m)
                print('-'*50)
                cph.print_summary()
        except Exception as e:
            print(e)
        
# results_df = pd.DataFrame({'feature':[],'modifier':[],'coef':[],'pval':[]})

/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_95578/4121709088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_95578/4121709088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_95578/4121709088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


delta contains nan value(s). Convergence halted. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model
delta contains nan value(s). Convergence halted. Please see the following tips in the lifelines documentation: https://lifelines.readthedocs.io/en/latest/Examples.html#problems-with-convergence-in-the-cox-proportional-hazard-model


/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_95578/4121709088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_95578/4121709088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]
/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_95578/4121709088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.


--------------------------------------------------
DEGREE * AGE7
--------------------------------------------------


/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_95578/4121709088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]


...

--------------------------------------------------
APOE_risk * CES-D_score
--------------------------------------------------


<lifelines.CoxPHFitter: fitted with 1229 total observations, 1038 right-censored observations>
             duration col = 'time_to_event'
                event col = 'event'
      baseline estimation = breslow
   number of observations = 1229
number of events observed = 191
   partial log-likelihood = -1246.80
         time fit was run = 2021-11-04 15:51:29 UTC

---
                   coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                
interaction_term   0.04       1.04       0.01             0.02             0.06                 1.02                 1.06

                    z      p   -log2(p)
covariate                              
interaction_term 4.53 <0.005      17.41
---
Concordance = 0.57
Partial AIC = 2495.59
log-likelihood ratio test = 17.71 on 1 df
-log2(p) of ll-ratio test = 15.25

/var/folders/v1/zs65yk6n05bc_3ty0jbr0ljsc7z4rq/T/ipykernel_95578/4121709088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_cox['interaction_term'] = df[f]*df[m]


--------------------------------------------------
APOE_risk * CSD-D_loneliness_item
--------------------------------------------------


<lifelines.CoxPHFitter: fitted with 1229 total observations, 1038 right-censored observations>
             duration col = 'time_to_event'
                event col = 'event'
      baseline estimation = breslow
   number of observations = 1229
number of events observed = 191
   partial log-likelihood = -1253.47
         time fit was run = 2021-11-04 15:51:29 UTC

---
                   coef  exp(coef)   se(coef)   coef lower 95%   coef upper 95%  exp(coef) lower 95%  exp(coef) upper 95%
covariate                                                                                                                
interaction_term   0.36       1.44       0.15             0.06             0.66                 1.06                 1.94

                    z    p   -log2(p)
covariate                            
interaction_term 2.36 0.02       5.79
---
Concordance = 0.53
Partial AIC = 2508.94
log-likelihood ratio test = 4.36 on 1 df
-log2(p) of ll-ratio test = 4.76

In [70]:
results_df = pd.concat(results)


In [76]:
results_df[results_df['term 1']=='AGE7']

,covariate,coef,exp(coef),se(coef),coef lower 95%,coef upper 95%,exp(coef) lower 95%,exp(coef) upper 95%,z,p,-log2(p),term 1,term 2
0,AGE7*SEX_SHANNON,0.004717,1.004728,0.002372,0.000068,0.009365,1.000068,1.009409,1.988813,4.672187e-02,4.419758,AGE7,SEX_SHANNON
0,AGE7*SEX_SIMPSON,0.004717,1.004728,0.002372,0.000068,0.009365,1.000068,1.009409,1.988813,4.672187e-02,4.419758,AGE7,SEX_SIMPSON
0,AGE7*RELTYPE_SHANNON,-0.001424,0.998577,0.002153,-0.005645,0.002796,0.994371,1.002800,-0.661466,5.083135e-01,0.976210,AGE7,RELTYPE_SHANNON
0,AGE7*RELTYPE_SIMPSON,-0.001424,0.998577,0.002153,-0.005645,0.002796,0.994371,1.002800,-0.661466,5.083135e-01,0.976210,AGE7,RELTYPE_SIMPSON
0,AGE7*ALTERTYPE_SHANNON,-0.002793,0.997211,0.002307,-0.007314,0.001729,0.992712,1.001731,-1.210426,2.261154e-01,2.144869,AGE7,ALTERTYPE_SHANNON
0,AGE7*ALTERTYPE_SIMPSON,-0.002793,0.997211,0.002307,-0.007314,0.001729,0.992712,1.001731,-1.210426,2.261154e-01,2.144869,AGE7,ALTERTYPE_SIMPSON
0,AGE7*AGE35_SHANNON,-0.000234,0.999766,0.002182,-0.004511,0.004044,0.995499,1.004052,-0.107155,9.146660e-01,0.128683,AGE7,AGE35_SHANNON
0,AGE7*AGE35_SIMPSON,-0.000234,0.999766,0.002182,-0.004511,0.004044,0.995499,1.004052,-0.107155,9.146660e-01,0.128683,AGE7,AGE35_SIMPSON
0,AGE7*AGE65_SHANNON,0.003531,1.003537,0.002170,-0.000723,0.007784,0.999278,1.007814,1.626998,1.037375e-01,3.268990,AGE7,AGE65_SHANNON
0,AGE7*AGE65_SIMPSON,0.003531,1.003537,0.002170,-0.000723,0.007784,0.999278,1.007814,1.626998,1.037375e-01,3.268990,AGE7,AGE65_SIMPSON


In [78]:
results_df.to_csv('modification_effects.csv')

In [79]:
res_df = results_df[results_df['p']<=.05][['term 1', 'term 2','exp(coef)','p']]


In [84]:
res_df[res_df['exp(coef)']>1.2]

,term 1,term 2,exp(coef),p
0,DEGREE_CENTRALITY,AGE7,2.725148e+01,2.496538e-03
0,DEGREE_CENTRALITY,APOE_risk,7.856598e+215,1.892827e-07
0,DEGREE_CENTRALITY,CVD_HISTORY,2.129612e+174,1.443102e-05
0,DEGREE_CENTRALITY,CES-D_score,6.728518e+04,2.549727e-02
0,DEGREE_CENTRALITY,CSD-D_loneliness_item,1.012930e+92,3.127851e-03
0,CONSTRAINT,APOE_risk,1.821925e+00,5.598774e-03
0,CONSTRAINT,CVD_HISTORY,1.761650e+00,2.521807e-02
0,CONSTRAINT,CSD-D_loneliness_item,1.761360e+00,1.225565e-03
0,EIGENVECTOR,APOE_risk,4.338587e+16,3.019328e-11
0,EIGENVECTOR,CVD_HISTORY,6.819092e+03,1.047151e-02
